In [2]:
# Import modules
import nibabel as nib
import pandas as pd
import os

In [3]:
nii_file = "C:/Users/smart/Desktop/read_dims/sub-C20_ses-001_run-02_T2w.nii.gz"

In [4]:
os.path.exists(nii_file)

True

In [5]:
img = nib.load(nii_file)

In [6]:
img

In [10]:
header = img.header
print(header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512  35   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.         0.313      0.313      3.0001009 19.376791   0.
  0.         0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'Time=0.000'
aux_file        : b'IRC287H'
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.45935443
quatern_c       : -0.5364548
quatern_d       : -0.48735195
qoffset_x       : -21.58

In [15]:
img.header['pixdim'][1:4]

array([0.313    , 0.313    , 3.0001009], dtype=float32)

In [16]:
def get_img_dims(nii_file):
    '''
    Ascertains image dimensions from input nifti image header.
    
    Arguments:
        nii_file (nifti file): NifTi image filename with absolute filepath
        
    Returns: 
        x (float): X dimension length (in mm).
        y (float): Y dimension length (in mm).
        z (float): Z dimension length (in mm).
    '''
    
    # Load image
    img = nib.load(nii_file)
    
    # Store image dimensions
    x = float(img.header['pixdim'][1])
    y = float(img.header['pixdim'][2])
    z = float(img.header['pixdim'][3])
    
    return x,y,z

In [30]:
[x,y,z] = get_img_dims(nii_file=nii_file)

In [31]:
x

0.31299999356269836

In [32]:
# z = y

In [33]:
round(x,1)

0.3

In [34]:
x == y == z

False

In [50]:
def determine_orientation(nii_file,num_dec=1):
    '''
    Determines image acquisition direction from input nifti image.
    
    Arguments:
        nii_file (nifti file): NifTi image filename with absolute filepath.
        num_dec (int): Number of decimal places to round each voxel dimension to.
        
    Returns: 
        orientation (string): Output string that describes image acquisition direction
        x (float): X dimension length (in mm).
        y (float): Y dimension length (in mm).
        z (float): Z dimension length (in mm).
    '''
    
    # Determine image dimensions
    [x,y,z] = get_img_dims(nii_file=nii_file)
    
    # Round values to managable numbers
    x = round(x,num_dec)
    y = round(y,num_dec)
    z = round(z,num_dec)
    
    # Ascertain image acquisition direction
    if x == y == z:
        orientation = "isotropic"
    elif x == y:
        orientation = "axial"
    elif x == z:
        orientation = "coronal"
    elif y == z:
        orientation = "sagittal"
    else:
        orientation = "inconsistent voxel sizes - review manually"
        
    return orientation,x,y,z

In [46]:
[o,x,y,z] = determine_orientation(nii_file=nii_file,num_dec=1)

In [54]:
def write_spread(nii_file,out_file,num_dec=1,full_path=False):
    '''
    Writes image filename, dimensions, and acquisition direction to a
    spreadsheet. If the spreadsheet already exists, then it is appended
    to.
    
    Arguments:
        nii_file (nifti file): NifTi image filename with absolute filepath.
        out_file (csv file): Output csv file name and path. This file need not exist at runtime.
        num_dec (int): Number of decimal places to round each voxel dimension to.
        full_path (bool): Whether to write the absolute path (vs the basename) in the spreadsheet.
            This option should be 'True' if all files in the spreadsheet are named similarly.
            Otherwise, should be 'False' if all the files are labeled uniquely (i.e. BIDS data).
        
    Returns: 
        out_file (csv file): Output csv file name and path.
    '''
    
    # Strip csv file extension from output file name
    if '.csv' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    elif '.tsv' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    elif '.txt' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    else:
        pass
    
    # Determine image orientation
    [orientation,x,y,z] = determine_orientation(nii_file=nii_file,num_dec=num_dec)
    
    # Construct image data dictionary
    if full_path:
        file = os.path.abspath(nii_file)
    else:
        file = os.path.basename(nii_file)
    
    img_dict = {"Filename":[file],
               "x_vox":[x],
               "y_vox":[y],
               "z_vox":[z],
               "Orientation":[orientation]}
    
    # Create dataframe from image dictionary
    df = pd.DataFrame.from_dict(img_dict,orient='columns')
    
    # Write output CSV file
    if os.path.exists(out_file):
        df.to_csv(out_file, sep=",", header=False, index=False, mode='a')
    else:
        df.to_csv(out_file, sep=",", header=True, index=False, mode='w')
    
    return out_file

In [52]:
os.path.splitext(nii_file)[0]

'C:/Users/smart/Desktop/read_dims/sub-C20_ses-001_run-02_T2w.nii'

In [48]:
t = "file"

In [49]:
os.path.splitext(t)

('file', '')

In [55]:
write_spread(nii_file=nii_file,out_file='test.csv',num_dec=1)

'test.csv'